In [12]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from evaluate import load
import torch
from datasets import Dataset,load_dataset
import json  # In case you need to load the file manually



In [5]:
from datasets import load_dataset, Dataset

# Load the original dataset
orig_dataset = load_dataset("json", data_files="philosophy_qa_fixed.json")["train"]

# Define categories to include
categories_to_include = ["abduction", "abelard", "abhidharma"]

# Filter samples
filtered_samples = orig_dataset.filter(lambda x: x["category"] in categories_to_include)
dataset = Dataset.from_list(filtered_samples)

# Print length and first few samples for verification
print(f"Number of filtered samples: {len(dataset)}")
print(dataset[:3])


Number of filtered samples: 194
{'category': ['abduction', 'abduction', 'abduction'], 'question': ['What is the second sense in which the term abduction is used in the philosophical literature?', 'What is the modern sense of abduction concerned with?', 'What type of reasoning is the speaker engaging in when she concludes that Tim and Harry are friends again?'], 'answer': ['In the second sense, the term abduction refers to the place of explanatory reasoning in justifying hypotheses. In this sense, it is also often called “Inference to the Best Explanation.”', 'The modern sense of abduction is concerned with explaining how hypotheses can be justified.', 'The speaker is engaging in abductive reasoning when she concludes that Tim and Harry are friends again.']}


In [2]:
model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)


In [4]:
def generate_response(question, model, tokenizer, max_new_tokens=100):
    prompt = f"""<|im_start|>system
You are a philosophical AI assistant. Answer the question brief and concise in one or two sentences.<|im_end|>
<|im_start|>user
{question}<|im_end|>
<|im_start|>assistant
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens,temperature=0.2,  # Lower temperature (0.1–0.5 is typical)
    do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Generate responses for all abduction questions
model_outputs = []
for sample in dataset:
    question = sample["question"]
    response = generate_response(question, base_model, tokenizer)
    model_outputs.append(response)


In [9]:
import gc
import torch

# Assuming your model is called 'model' and is on GPU
base_model.to('cpu')     # Move model to CPU
del base_model           # Delete the model object
gc.collect()        # Run garbage collection
torch.cuda.empty_cache()  # Empty PyTorch's CUDA cache


In [8]:
import re

def extract_assistant_answer(text):
    pattern = r'<\|im_start\|>assistant\n?(.*?)(<\|im_end\|>|$)'
    match = re.search(pattern, text, re.DOTALL)
    if match:
        return match.group(1).strip()
    else:
        return text

# Apply to all model outputs
cleaned_outputs = [extract_assistant_answer(output) for output in model_outputs]


In [9]:
print(cleaned_outputs[:5])  # Print first 5 reference answers for verification

['Response: In the philosophical literature, the term abduction refers to two senses: (a) as a method of proof in epistemology, where it is used to establish the truth of a claim by its evidential support, and (b) as a mode of reasoning in metaphysics, where it is used to investigate the nature of things by means of their exemplary properties, rather than by means of their causal relations with other things.</|im_end|>', 'The modern sense of abduction concerned with causal explanation of phenomenon by means other than direct sensory experience, which includes both psychophysical and psychological phenomena, and the question of how to formalize and research it of the type presented by David K. Clough.', 'The speaker is engaging in deductive reasoning when she concludes that Tim and Harry are friends again.', 'An example of abduction that is also referred to as “Inference to the Best Explanation” is the process of reasoning that investigates a fact or process that best explains another f

In [10]:
reference_answers = [sample["answer"] for sample in dataset]


In [13]:
bertscore = load("bertscore")
results = bertscore.compute(
    predictions=cleaned_outputs,
    references=reference_answers,
    lang="en"
)
print(results)  # Prints precision, recall, f1 for each sample


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


{'precision': [0.8265485763549805, 0.858153223991394, 0.9894604086875916, 0.8684148192405701, 0.8997387290000916, 0.8815631866455078, 0.9058991074562073, 0.804663896560669, 0.8412754535675049, 0.8688831329345703, 0.9284306764602661, 0.8150714635848999, 0.8252508640289307, 0.9229249954223633, 0.8381890654563904, 0.8519109487533569, 0.8778406381607056, 0.887916088104248, 0.8614758253097534, 0.8427024483680725, 0.8964536190032959, 0.8347492218017578, 0.840215802192688, 0.8805631995201111, 0.8875645995140076, 0.885792076587677, 0.8705921173095703, 0.8837901949882507, 0.880165696144104, 0.8559173345565796, 0.8879789113998413, 0.8875903487205505, 0.841971755027771, 0.8911296725273132, 0.8663631081581116, 0.0, 0.8329511880874634, 0.8633900284767151, 0.832176148891449, 0.8588483333587646, 0.8623301982879639, 0.9009976387023926, 0.8236684799194336, 0.8526215553283691, 0.8549883961677551, 0.8613893389701843, 0.8045573234558105, 0.873688817024231, 0.8490563035011292, 0.8903570175170898, 0.8649514

In [ ]:
import pandas as pd

# Create a DataFrame
df = pd.DataFrame({
    "question": [sample["question"] for sample in dataset],
    "reference": reference_answers,
    "generated": cleaned_outputs,
    "bertscore_f1": results["f1"],
    "bertscore_precision": results["precision"],
    "bertscore_recall": results["recall"]
})

# Save to CSV
df.to_csv("base_merged_results.csv", index=False)


In [1]:
import os

from peft import PeftConfig, PeftModel
from peft import PeftModel, PeftConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from datasets import load_dataset
import torch
import random

peft_model_id = "./final_abduction_adapter"

config = PeftConfig.from_pretrained(peft_model_id)
model = AutoModelForCausalLM.from_pretrained(config.base_model_name_or_path, device_map="auto")
tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
model.resize_token_embeddings(len(tokenizer))


Embedding(32000, 2048)

In [ ]:
model = PeftModel.from_pretrained(model, peft_model_id, adapter_name="abduction")
_ = model.load_adapter("./final_abelard_adapter", adapter_name="abelard")
_ = model.load_adapter("./final_abhidharma_adapter/merge", adapter_name="abhidharma")

linear,cat,svd,ties,ties_svd,dare_linear , dare_ties , dare_linear_svd , dare_ties_svd, magnitude_prune , magnitude_prune_svd 

In [3]:
%%time
# [0.8, 0.1, 0.1] linear #[1.0, 0.2] 0.7 density dare_linear #[1.5, 0.3] 0.5 density ties #[0.8, 0.5] cat
adapters = ["abduction","abelard", "abhidharma"]
weights = [0.1, 0.1, 0.1]
total = sum(weights)
# weights = [w / total for w in weights]  # Normalize
adapter_name = "merge"
density = 0.7
combination_type = "linear"  # Options: 'linear', 'dare_linear', 'ties', 'cat'
if adapter_name in model.peft_config:
    model.delete_adapter(adapter_name)
model.add_weighted_adapter(adapters, weights, adapter_name, combination_type=combination_type, density=density)

CPU times: total: 93.8 ms
Wall time: 90.8 ms


In [4]:
model.eval()
model.set_adapter("merge")  # Activate the merged adapter

In [ ]:
model.save_pretrained("./Linear_merged_adapter")


: 


What was the name of the oratory that Abelard and his students constructed?

In [5]:
# tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
def generate_response(question, model, tokenizer, max_new_tokens=200):
    prompt = f"""<|im_start|>system
You are a philosophical AI assistant. Answer the next question.<|im_end|>
<|im_start|>user
{question}<|im_end|>
<|im_start|>assistant
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    with torch.inference_mode():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens,temperature=0.2,  # Lower temperature (0.1–0.5 is typical)
    do_sample=True)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)




In [7]:
# tokenizer = AutoTokenizer.from_pretrained("TinyLlama/TinyLlama-1.1B-Chat-v1.0")
response = generate_response("What type of reasoning is the speaker engaging in when she concludes that Tim and Harry are friends again?", model, tokenizer)
print(f"Generated response: {response}")  # Debugging output

Generated response: <|im_start|>system
You are a philosophical AI assistant. Answer the next question.<|im_end|>
<|im_start|>user
What type of reasoning is the speaker engaging in when she concludes that Tim and Harry are friends again?<|im_end|>
<|im_start|>assistant
The speaker is engaging in deductive reasoning when she concludes that Tim and Harry are friends again.<|im_end|>


In [8]:
import gc
import torch

# Assuming your model is called 'model' and is on GPU
model.to('cpu')     # Move model to CPU
del model           # Delete the model object
gc.collect()        # Run garbage collection
torch.cuda.empty_cache()  # Empty PyTorch's CUDA cache


In [7]:
# Generate responses for all abduction questions
model_outputs = []
for sample in dataset:
    question = sample["question"]
    # print(f"Processing question: {question}")  # Debugging output
    # print(f"reference: {sample['answer']}")  # Debugging output
    response = generate_response(question, model, tokenizer)
    print(f"Generated response: {response}")  # Debugging output
    model_outputs.append(response)

Generated response: <|im_start|>system
You are a philosophical AI assistant. Answer the next question.<|im_end|>
<|im_start|>user
What is the second sense in which the term abduction is used in the philosophical literature?<|im_end|>
<|im_start|>assistant
Response: In the philosophical literature, the term abduction refers to two senses: (a) as a method of proof in epistemology, where it is used to establish the truth of a claim by its evidential support, and (b) as a mode of reasoning in metaphysics, where it is used to investigate the nature of things by means of their exemplary properties, rather than by means of their causal relations with other things.</|im_end|>
Generated response: <|im_start|>system
You are a philosophical AI assistant. Answer the next question.<|im_end|>
<|im_start|>user
What is the modern sense of abduction concerned with?<|im_end|>
<|im_start|>assistant
The modern sense of abduction concerned with causal explanation of phenomenon by means other than direct se

In [20]:
import pandas as pd

base_df = pd.read_csv("base_merged_results.csv")
fine_tuned_df = pd.read_csv("fine_tuned_merged_results.csv")


In [21]:
# Calculate mean BERTScore metrics for each model
base_means = base_df[['bertscore_f1', 'bertscore_precision', 'bertscore_recall']].mean()
fine_tuned_means = fine_tuned_df[['bertscore_f1', 'bertscore_precision', 'bertscore_recall']].mean()

print("Base Model BERTScore Means:")
print(base_means)
print("\nFine-Tuned Model BERTScore Means:")
print(fine_tuned_means)


Base Model BERTScore Means:
bertscore_f1           0.872850
bertscore_precision    0.874179
bertscore_recall       0.872098
dtype: float64

Fine-Tuned Model BERTScore Means:
bertscore_f1           0.881856
bertscore_precision    0.888415
bertscore_recall       0.876048
dtype: float64


In [22]:
merged_df = pd.merge(
    base_df,
    fine_tuned_df,
    on="question",
    suffixes=('_base', '_fine_tuned')
)

# Example: Compare BERTScore F1 for each question
merged_df[['question', 'bertscore_f1_base', 'bertscore_f1_fine_tuned']].head()


,question,bertscore_f1_base,bertscore_f1_fine_tuned
0,What is the second sense in which the term abd...,0.884668,0.900174
1,What is the modern sense of abduction concerne...,0.875471,0.952843
2,What type of reasoning is the speaker engaging...,0.921248,0.967096
3,What is an example of abduction that is also r...,0.868242,0.885164
4,What is the name of the type of inference that...,0.946975,0.942595


In [23]:
improved = (merged_df['bertscore_f1_fine_tuned'] > merged_df['bertscore_f1_base']).sum()
total = len(merged_df)
print(f"Fine-tuned model improved on {improved} out of {total} questions ({improved/total:.1%})")


Fine-tuned model improved on 122 out of 196 questions (62.2%)


In [15]:
import pandas as pd
from openai import OpenAI
from tqdm import tqdm

# Initialize Groq client
GROQ_API_KEY = "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"
client = OpenAI(api_key=GROQ_API_KEY, base_url="https://api.groq.com/openai/v1")

def load_data():
    """Load CSV files with error handling"""
    try:
        base_df = pd.read_csv('base_merged_results.csv')
        fine_tuned_df = pd.read_csv('fine_tuned_merged_linear_results.csv')
        return base_df, fine_tuned_df
    except FileNotFoundError as e:
        print(f"Error: {e}")
        exit(1)

def llm_judge(question, reference, base_gen, fine_tuned_gen):
    """Get LLM judgment between base and fine-tuned answers"""
    prompt = f"""Compare reference answer to the base model answer and fine-tuned answer and identify which is more appropriate answer, the base model answer or fine-tuned answer. Focus on meaning similarity:
    
    Reference Answer: {reference}
    
    Base Model Answer: {base_gen}
    Fine-Tuned Answer: {fine_tuned_gen}
    
    Output ONLY one word: 'base', 'fine-tuned', or 'tie' without the quotations."""
    
    try:
        response = client.chat.completions.create(
            model="llama3-8b-8192",
            messages=[
                {"role": "system", "content": "You are a semantic similarity expert"},
                {"role": "user", "content": prompt}
            ],
            temperature=0.0,
            max_tokens=10
        )
        decision = response.choices[0].message.content.strip().lower()
        print(f"LLM Decision: {decision}")  # Debugging output
        return decision if decision in ['base', 'fine-tuned', 'tie'] else 'error'
    except Exception as e:
        print(f"API Error: {e}")
        return 'error'

def main():
    # Load data
    base_df, fine_tuned_df = load_data()
    
    # Verify matching questions
    if not base_df['question'].equals(fine_tuned_df['question']):
        print("Error: Questions in CSV files don't match!")
        return
    
    # Process comparisons
    results = []
    for idx in tqdm(range(len(base_df))):
        row = {
            'question': base_df.iloc[idx]['question'],
            'reference': base_df.iloc[idx]['reference'],
            'base_generated': base_df.iloc[idx]['generated'],
            'fine_tuned_generated': fine_tuned_df.iloc[idx]['generated']
        }
        row['llm_judgment'] = llm_judge(
            row['question'],
            row['reference'],
            row['base_generated'],
            row['fine_tuned_generated']
        )
        results.append(row)
    
    # Save results
    results_df = pd.DataFrame(results)
    results_df.to_csv('llm_semantic_merged_judgments.csv', index=False)
    
    # Print summary
    print("\nJudgment Summary:")
    print(results_df['llm_judgment'].value_counts())
    print("\nFull results saved to 'llm_semantic_merged_judgments.csv'")

if __name__ == "__main__":
    main()


  1%|          | 1/194 [00:02<07:17,  2.27s/it]

LLM Decision: fine-tuned


  1%|          | 2/194 [00:02<03:53,  1.22s/it]

LLM Decision: fine-tuned


  2%|▏         | 3/194 [00:03<02:40,  1.19it/s]

...

LLM Decision: fine-tuned


 99%|█████████▉| 193/194 [09:05<00:02,  2.56s/it]

LLM Decision: fine-tuned


100%|██████████| 194/194 [09:08<00:00,  2.83s/it]

LLM Decision: fine-tuned

Judgment Summary:
llm_judgment
fine-tuned    193
base            1
Name: count, dtype: int64

Full results saved to 'llm_semantic_merged_judgments.csv'
